In [29]:
import re
import spacy
from spacy.matcher import Matcher
import fitz
import os


In [30]:
# Descargar el modelo si no está instalado
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    from spacy.cli import download
    download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

matcher = Matcher(nlp.vocab)

In [31]:
def extraer_datos_gaceta(ruta_pdf):
    """
    Extrae datos de una gaceta del Congreso usando spaCy y expresiones regulares.
    """
    datos = {}

    try:
        with fitz.open(ruta_pdf) as doc:
            texto_completo = ""
            for pagina in doc:
                texto_completo += pagina.get_text()
                
            
                        # --- 1. Expresiones Regulares ---
            texto_completo = preprocesar_texto(texto_completo)
            datos["numero_gaceta"] = extraer_con_regex(r"AÑO\s+(.*?)\s+N.°\s+(\d+)", texto_completo) #se agrega año para mayor contexto
            datos["issn"] = extraer_con_regex(r"ISSN\s+(\d{4}\s*-\s*\d+)", texto_completo)  # Permite espacios entre los dígitos
            datos["fecha_publicacion"] = extraer_con_regex(r"Bogotá,\s+D\.C\.,\s+(?:.*?),\s*(\d+\s+de\s+[a-zA-Z]+\s+de\s+\d+)", texto_completo)
            datos["num_paginas"] = extraer_con_regex(r"EDICIÓN\s+DE\s+(\d+)\s+PÁGINAS", texto_completo)
# ... (otras expresiones regulares para "Número de Cámara" y "Número de Senado")
            # --- 2. spaCy ---
            doc_spacy = nlp(texto_completo)

            # Directores y secretarios (Matcher)
            patron_directores = [{"LOWER": "directores"}, {"POS": "PROPN", "OP": "+"}]
            matcher.add("DIRECTORES", [patron_directores])
            matches = matcher(doc_spacy)
            directores = []
            for match_id, start, end in matches:
                span = doc_spacy[start:end]
                directores.append(span.text)
            datos["directores_secretarios"] = ", ".join(directores[1:])

            # Nombre del proyecto de ley (Matcher o análisis de oraciones)
            # Define patrones o reglas para encontrar el nombre del proyecto
            # ...
            for oracion in doc_spacy.sents:
                if 'COMENTARIOS AL PROYECTO DE LEY' in oracion.text.upper():
                    nombre_proyecto = oracion.text.upper()
                    nombre_proyecto = nombre_proyecto.split("COMENTARIOS AL PROYECTO DE LEY")[1].strip()
                    datos["nombre_proyecto"] = nombre_proyecto.title()  # nombre del proyecto ley
                    break

            # Palabras clave (NER de spaCy)
            palabras_clave = [ent.text for ent in doc_spacy.ents if ent.label_ == "MISC"]  # MISC para entidades diversas
            datos["palabras_claves"] = ", ".join(palabras_clave)

            # --- 3. Reglas Heurísticas (basadas en el PDF de descripción) ---
            # Título de la Gaceta (si siempre está en la primera línea o en un lugar específico)
            lineas = [linea for linea in texto_completo.strip().splitlines() if linea.strip()] #se filtran lineas vacias
            datos["titulo_gaceta"] = lineas[0].strip()
            
            
    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo PDF: {ruta_pdf}")
        return None
    except Exception as e:
        print(f"Error al procesar el PDF {ruta_pdf}: {e}")
        return None

    return datos                        

In [32]:
def extraer_con_regex(patron, texto):
    coincidencia = re.search(patron, texto, re.IGNORECASE) #se agrega ignorecase para evitar problemas con mayusculas y minusculas
    if coincidencia:
        return coincidencia.group(1).replace(" ", "") #se eliminan espacios
    return None

In [33]:

def preprocesar_texto(texto):

    texto = texto.replace("-\n", "") #une palabras separadas por guion y salto de linea
    texto = texto.replace("\n", " ") #reemplaza saltos de linea por espacios
    texto = re.sub(r"\s+", " ", texto) #elimina espacios dobles
    texto = texto.strip()
    return texto





In [34]:

# --- Ejemplo de Uso ---
ruta_gaceta = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\20160328 XXV 110_64.pdf"  # Reemplaza con la ruta de tu PDF
datos_extraidos = extraer_datos_gaceta(ruta_gaceta)


if datos_extraidos:
    print(datos_extraidos)

{'numero_gaceta': None, 'issn': None, 'fecha_publicacion': None, 'num_paginas': '64', 'directores_secretarios': '', 'nombre_proyecto': 'Número 97 De 2015 Senado “Por La Cual Se Prohíbe La Producción, Comercialización, Exportación, Importación Y Distribución De Cualquier Variedad De Asbesto En Colombia”.', 'palabras_claves': 'Página 1 C O M E N TA R I O S, Artículo 36, Ley, IMPRENTA NACIONAL DE COLOMBIA www.imprenta.gov.co SENADO, CÁMARA AÑO XXV, EDICIÓN, 64 PÁGINAS DIRECTORES, REPÚBLICA, CÁMARA www.camara.gov.co GREGORIO ELJACH, SECRETARIO, SENADO www.secretariasenado.gov.co G A C E T A D E L C O N G R E S O I S S N, PROYECTO, El término “, Los seis minerales comúnmente, Un grupo, CRISOTILO El crisotilo, La hoja, Si la ﬁbra, Crisotilo Figura 2 - Desintegración de la Fibra, Crisotilo Agosto, Ley, IMPRENTA NACIONAL DE COLOMBIA www, gov.co, EDICION, DIRECTORES, SE RETARIO, SEN DO, CAMARA, W, gov.co WWW, co RAMA LEGISLATIVA, PROYECTO, CRISOTILO Agosto 2009, El término asbesto, Los seis min